In [1]:
!pip install optuna

  Using cached optuna-3.1.0-py3-none-any.whl (365 kB)
  Using cached cmaes-0.9.1-py3-none-any.whl (21 kB)
  Using cached colorlog-6.7.0-py2.py3-none-any.whl (11 kB)


In [ ]:
#
import optuna
from sklearn.metrics import mean_squared_error
import xarray as xr
import math
import stackstac
import datetime

import rasterio
from rasterio import windows
from rasterio import features as ft
from rasterio import warp
from PIL import Image

from matplotlib.pyplot import figure

# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Visualization
import ipyleaflet
import matplotlib.pyplot as plt
from IPython.display import Image
import seaborn as sns

# Data Science
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Feature Engineering
from sklearn.model_selection import train_test_split

# Machine Learning
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import r2_score


# Planetary Computer Tools
import pystac
import pystac_client
import odc
from pystac_client import Client
from pystac.extensions.eo import EOExtension as eo
from odc.stac import stac_load
import planetary_computer as pc

# Others
import requests
import rich.table
from itertools import cycle
from tqdm import tqdm
tqdm.pandas()
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

In [ ]:
crop_yield_data = pd.read_csv("Crop_Yield_Data_challenge_2.csv")
crop_yield_data.head()

,District,Latitude,Longitude,"Season(SA = Summer Autumn, WS = Winter Spring)","Rice Crop Intensity(D=Double, T=Triple)",Date of Harvest,Field size (ha),Rice Yield (kg/ha)
0,Chau_Phu,10.510542,105.248554,SA,T,15-07-2022,3.40,5500
1,Chau_Phu,10.509150,105.265098,SA,T,15-07-2022,2.43,6000
2,Chau_Phu,10.467721,105.192464,SA,D,15-07-2022,1.95,6400
3,Chau_Phu,10.494453,105.241281,SA,T,15-07-2022,4.30,6000
4,Chau_Phu,10.535058,105.252744,SA,D,14-07-2022,3.30,6400


## Get Sentinel 1 Data

In [32]:
def get_sentinel_data(longitude, latitude, season, crop_intensity, date_of_harvest, assets):
    
    '''
    Returns a list of VV,VH, VV/VH values for a given latitude and longitude over a given time period (based on the crop cycle)
    Attributes:
    longitude - Longitude
    latitude - Latitude
    season - The season for which band values need to be extracted (SA or WS)
    crop_intensity - The intensity of rice crop (D for Double, T for Triple)
    date_of_harvest - The date of harvest in format 'YYYY-MM-DD'
    assets - A list of bands to be extracted
    
    '''
    
    bands_of_interest = assets
    
        # Define the crop cycle start and end dates based on the season and crop intensity
    if season == 'SA':
        if crop_intensity == 'D':
            crop_start_date = datetime.datetime.strptime("2022-05-01", "%Y-%m-%d")
            crop_end_date = datetime.datetime.strptime("2022-08-31", "%Y-%m-%d")
        elif crop_intensity == 'T':
            crop_start_date = datetime.datetime.strptime("2022-06-01", "%Y-%m-%d")
            crop_end_date = datetime.datetime.strptime("2022-09-30", "%Y-%m-%d")
    elif season == 'WS':
        if crop_intensity == 'D':
            crop_start_date = datetime.datetime.strptime("2021-11-01", "%Y-%m-%d")
            crop_end_date = datetime.datetime.strptime("2022-04-30", "%Y-%m-%d")
        elif crop_intensity == 'T':
            crop_start_date = datetime.datetime.strptime("2021-12-01", "%Y-%m-%d")
            crop_end_date = datetime.datetime.strptime("2022-05-31", "%Y-%m-%d")

    # Parse the date_of_harvest string into a datetime object
    date_of_harvest = datetime.datetime.strptime(date_of_harvest, "%d-%m-%Y")

    # Define the time slice based on the crop cycle start and end dates and the date of harvest
    if date_of_harvest < crop_start_date:
        time_slice = f"{date_of_harvest.strftime('%Y-%m-%d')}/{crop_start_date.strftime('%Y-%m-%d')}"
    elif date_of_harvest > crop_end_date:
        time_slice = f"{crop_end_date.strftime('%Y-%m-%d')}/{date_of_harvest.strftime('%Y-%m-%d')}"
    else:
        time_slice = f"{crop_start_date.strftime('%Y-%m-%d')}/{crop_end_date.strftime('%Y-%m-%d')}"

        
    vv_list = []
    vh_list = []
    vv_by_vh_list = []
    
    bbox_of_interest = [longitude , latitude, longitude, latitude]
    time_of_interest = time_slice
    
    catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
    search = catalog.search(collections=["sentinel-1-rtc"], bbox=bbox_of_interest, datetime=time_of_interest)
    items = list(search.get_all_items())
    item = items[0]
    items.reverse()
    
    data = stac_load([items[1]],bands=bands_of_interest, patch_url=pc.sign, bbox=bbox_of_interest).isel(time=0)

    for item in items:
        data = stac_load([item], bands=bands_of_interest, patch_url=pc.sign, bbox=bbox_of_interest).isel(time=0)
        if(data['vh'].values[0][0]!=-32768.0 and data['vv'].values[0][0]!=-32768.0):
            data = data.where(~data.isnull(), 0)
            vh = data["vh"].astype("float64")
            vv = data["vv"].astype("float64")
            vv_list.append(np.median(vv))
            vh_list.append(np.median(vh))
            vv_by_vh_list.append(np.median(vv)/np.median(vh))
              
    return vv_list, vh_list, vv_by_vh_list

In [34]:
## Get Sentinel-1-RTC Data

num_parts = 10
train_band_values_parts = np.array_split(crop_yield_data, num_parts)


assets = ['vh','vv']
results = []

for part in train_band_values_parts:
    part_results = part.apply(lambda x: get_sentinel_data(x['Longitude'], x['Latitude'],x['Season(SA = Summer Autumn, WS = Winter Spring)'],x['Rice Crop Intensity(D=Double, T=Triple)'],x['Date of Harvest'],assets), axis=1)
    results.append(part_results)

# Concatenate the results
vh = []
vv = []
vv_by_vh = []

for r in results:
    vh += [x[0] for x in r]
    vv += [x[1] for x in r]
    vv_by_vh += [x[2] for x in r]

vh_vv_data = pd.DataFrame(list(zip(vh,vv,vv_by_vh)),columns = ["vv_list","vh_list","vv/vh_list"])


## Main Features

In [35]:
def ordinal_distribution(data, dx=3, dy=1, taux=1, tauy=1, return_missing=False, tie_precision=None):
    '''
    Returns
    -------
     : tuple
       Tuple containing two arrays, one with the ordinal patterns occurring in data 
       and another with their corresponding probabilities.
       
    Attributes
    ---------
    data : array 
           Array object in the format :math:`[x_{1}, x_{2}, x_{3}, \\ldots ,x_{n}]`
           or  :math:`[[x_{11}, x_{12}, x_{13}, \\ldots, x_{1m}],
           \\ldots, [x_{n1}, x_{n2}, x_{n3}, \\ldots, x_{nm}]]`.
    dx : int
         Embedding dimension (horizontal axis) (default: 3).
    dy : int
         Embedding dimension (vertical axis); it must be 1 for time series 
         (default: 1).
    taux : int
           Embedding delay (horizontal axis) (default: 1).
    tauy : int
           Embedding delay (vertical axis) (default: 1).
    return_missing: boolean
                    If `True`, it returns ordinal patterns not appearing in the 
                    symbolic sequence obtained from **data** are shown. If `False`,
                    these missing patterns (permutations) are omitted 
                    (default: `False`).
    tie_precision : int
                    If not `None`, **data** is rounded with `tie_precision`
                    number of decimals (default: `None`).
   
    '''
    def setdiff(a, b):
        '''
        Returns
        -------
        : array
            An array containing the elements in `a` that are not contained in `b`.
            
        Parameters
        ----------    
        a : tuples, lists or arrays
            Array in the format :math:`[[x_{21}, x_{22}, x_{23}, \\ldots, x_{2m}], 
            \\ldots, [x_{n1}, x_{n2}, x_{n3}, ..., x_{nm}]]`.
        b : tuples, lists or arrays
            Array in the format :math:`[[x_{21}, x_{22}, x_{23}, \\ldots, x_{2m}], 
            \\ldots, [x_{n1}, x_{n2}, x_{n3}, ..., x_{nm}]]`.
        '''

        a = np.asarray(a).astype('int64')
        b = np.asarray(b).astype('int64')

        _, ncols = a.shape

        dtype={'names':['f{}'.format(i) for i in range(ncols)],
            'formats':ncols * [a.dtype]}

        C = np.setdiff1d(a.view(dtype), b.view(dtype))
        C = C.view(a.dtype).reshape(-1, ncols)

        return(C)

    try:
        ny, nx = np.shape(data)
        data   = np.array(data)
    except:
        nx     = np.shape(data)[0]
        ny     = 1
        data   = np.array([data])

    if tie_precision is not None:
        data = np.round(data, tie_precision)

    partitions = np.concatenate(
        [
            [np.concatenate(data[j:j+dy*tauy:tauy,i:i+dx*taux:taux]) for i in range(nx-(dx-1)*taux)] 
            for j in range(ny-(dy-1)*tauy)
        ]
    )

    symbols = np.apply_along_axis(np.argsort, 1, partitions)
    symbols, symbols_count = np.unique(symbols, return_counts=True, axis=0)

    probabilities = symbols_count/len(partitions)

    if return_missing==False:
        return symbols, probabilities
    
    else:
        all_symbols   = list(map(list,list(itertools.permutations(np.arange(dx*dy)))))
        miss_symbols  = setdiff(all_symbols, symbols)
        symbols       = np.concatenate((symbols, miss_symbols))
        probabilities = np.concatenate((probabilities, np.zeros(miss_symbols.__len__())))
        
        return symbols, probabilities

In [36]:
def permutation_entropy(data, dx=3, dy=1, taux=1, tauy=1, base=2, normalized=True, probs=False, tie_precision=None):
    '''
    Returns Permutation Entropy
    Attributes:
    data : array
           Array object in the format :math:`[x_{1}, x_{2}, x_{3}, \\ldots ,x_{n}]`
           or  :math:`[[x_{11}, x_{12}, x_{13}, \\ldots, x_{1m}],
           \\ldots, [x_{n1}, x_{n2}, x_{n3}, \\ldots, x_{nm}]]`
           or an ordinal probability distribution (such as the ones returned by :func:`ordpy.ordinal_distribution`).
    dx :   int
           Embedding dimension (horizontal axis) (default: 3).
    dy :   int
           Embedding dimension (vertical axis); it must be 1 for time series (default: 1).
    taux : int
           Embedding delay (horizontal axis) (default: 1).
    tauy : int
           Embedding delay (vertical axis) (default: 1).
    base : str, int
           Logarithm base in Shannon's entropy. Either 'e' or 2 (default: 2).
    normalized: boolean
                If `True`, permutation entropy is normalized by its maximum value 
                (default: `True`). If `False`, it is not.
    probs : boolean
            If `True`, assumes **data** is an ordinal probability distribution. If 
            `False`, **data** is expected to be a one- or two-dimensional 
            array (default: `False`). 
    tie_precision : int
                    If not `None`, **data** is rounded with `tie_precision`
                    number of decimals (default: `None`).
    '''
    if not probs:
        _, probabilities = ordinal_distribution(data, dx, dy, taux, tauy, return_missing=False, tie_precision=tie_precision)
    else:
        probabilities = np.asarray(data)
        probabilities = probabilities[probabilities>0]

    if normalized==True and base in [2, '2']:        
        smax = np.log2(float(np.math.factorial(dx*dy)))
        s    = -np.sum(probabilities*np.log2(probabilities))
        return s/smax
         
    elif normalized==True and base=='e':        
        smax = np.log(float(np.math.factorial(dx*dy)))
        s    = -np.sum(probabilities*np.log(probabilities))
        return s/smax
    
    elif normalized==False and base in [2, '2']:
        return -np.sum(probabilities*np.log2(probabilities))
    else:
        return -np.sum(probabilities*np.log(probabilities))

In [37]:
def generate_stastical_features(dataframe):
    '''
    Returns a  list of statistical features such as min,max,range,mean,auto-correlation,permutation entropy for each of the features
    Attributes:
    dataframe - DataFrame consisting of VV,VH and VV/VH for a time period
    '''
    features_list = []
    for index, row in dataframe.iterrows():

        min_vv = min(row[0])
        max_vv = max(row[0])
        range_vv = max_vv - min_vv
        mean_vv = np.mean(row[0])
        correlation_vv = sm.tsa.acf(row[0])[1]
        permutation_entropy_vv = permutation_entropy(row[0], dx=6,base=2, normalized=True)    
    
        min_vh = min(row[1])
        max_vh = max(row[1])
        range_vh = max_vh - min_vh
        mean_vh = np.mean(row[1])
        correlation_vh = sm.tsa.acf(row[1])[1]
        permutation_entropy_vh = permutation_entropy(row[1], dx=6, base=2, normalized=True)
    
        min_vv_by_vh = min(row[2])
        max_vv_by_vh = max(row[2])
        range_vv_by_vh = max_vv_by_vh - min_vv_by_vh
        mean_vv_by_vh = np.mean(row[2])
        correlation_vv_by_vh = sm.tsa.acf(row[2])[1]
        permutation_entropy_vv_by_vh = permutation_entropy(row[2], dx=6, base=2, normalized=True)
        
        ''
        min_rvi = math.sqrt (1- min_vv / (min_vv + min_vh)) * 4 * (min_vh / (min_vv + min_vh))
        max_rvi = math.sqrt (1- max_vv / (max_vv + max_vh)) * 4 * (max_vh / (max_vv + max_vh))
        range_rvi = math.sqrt (1- range_vv / (range_vv + range_vh)) * 4 * (range_vh / (range_vv + range_vh))
        mean_rvi = math.sqrt (1- mean_vv / (mean_vv + mean_vh)) * 4 * (mean_vh / (mean_vv + mean_vh))
        ''
    
        features_list.append([min_vv, max_vv, range_vv, mean_vv, correlation_vv, permutation_entropy_vv,
                          min_vh, max_vh, range_vh,  mean_vh, correlation_vh, permutation_entropy_vh,
                          min_vv_by_vh,  max_vv_by_vh, range_vv_by_vh, mean_vv_by_vh, correlation_vv_by_vh, permutation_entropy_vv_by_vh,
                             ##min_rvi,max_rvi,range_rvi,mean_rvi
                             ])
    
    columns=['min_vv', 'max_vv', 'range_vv', 'mean_vv', 'correlation_vv', 'permutation_entropy_vv',
            'min_vh', 'max_vh', 'range_vh', 'mean_vh', 'correlation_vh', 'permutation_entropy_vh',
            'min_vv_by_vh',  'max_vv_by_vh', 'range_vv_by_vh', 'mean_vv_by_vh', 'correlation_vv_by_vh', 'permutation_entropy_vv_by_vh',
            ##'min_rvi','max_rvi','range_rvi','mean_rvi'
            ]
    features_df = pd.DataFrame(features_list, columns=columns)
    
    
    return features_df

In [38]:
def generate_stastical_features_original(dataframe):
    '''
    Returns a  list of statistical features such as min,max,range,mean,auto-correlation,permutation entropy for each of the features
    Attributes:
    dataframe - DataFrame consisting of VV,VH and VV/VH for a time period
    '''
    features_list = []
    for index, row in dataframe.iterrows():

        min_vv = min(row[0])
        max_vv = max(row[0])
        range_vv = max_vv - min_vv
        mean_vv = np.mean(row[0])
        correlation_vv = sm.tsa.acf(row[0])[1]
        permutation_entropy_vv = permutation_entropy(row[0], dx=6,base=2, normalized=True) 
    
        min_vh = min(row[1])
        max_vh = max(row[1])
        range_vh = max_vh - min_vh
        mean_vh = np.mean(row[1])
        correlation_vh = sm.tsa.acf(row[1])[1]
        permutation_entropy_vh = permutation_entropy(row[1], dx=6, base=2, normalized=True)
    
        min_vv_by_vh = min(row[2])
        max_vv_by_vh = max(row[2])
        range_vv_by_vh = max_vv_by_vh - min_vv_by_vh
        mean_vv_by_vh = np.mean(row[2])
        correlation_vv_by_vh = sm.tsa.acf(row[2])[1]
        permutation_entropy_vv_by_vh = permutation_entropy(row[2], dx=6, base=2, normalized=True)
        
        '''
        min_rvi = math.sqrt (1- min_vv / (min_vv + min_vh)) * 4 * (min_vh / (min_vv + min_vh))
        max_rvi = math.sqrt (1- max_vv / (max_vv + max_vh)) * 4 * (max_vh / (max_vv + max_vh))
        range_rvi = math.sqrt (1- range_vv / (range_vv + range_vh)) * 4 * (range_vh / (range_vv + range_vh))
        mean_rvi = math.sqrt (1- mean_vv / (mean_vv + mean_vh)) * 4 * (mean_vh / (mean_vv + mean_vh))
        '''
    
        features_list.append([min_vv, max_vv, range_vv, mean_vv, correlation_vv, permutation_entropy_vv,
                          min_vh, max_vh, range_vh,  mean_vh, correlation_vh, permutation_entropy_vh,
                          min_vv_by_vh,  max_vv_by_vh, range_vv_by_vh, mean_vv_by_vh, correlation_vv_by_vh, permutation_entropy_vv_by_vh,
                              ##min_rvi,max_rvi,range_rvi,mean_rvi
                             ])

    
    return features_list

In [77]:
# Generating Statistical Features for VV,VH and VV/VH and creating a dataframe
features = generate_stastical_features_original(vh_vv_data)
features_data = pd.DataFrame(features ,columns = ['min_vv', 'max_vv', 'range_vv', 'mean_vv', 'correlation_vv', 'permutation_entropy_vv',
                          'min_vh', 'max_vh', 'range_vh', 'mean_vh', 'correlation_vh', 'permutation_entropy_vh',
                          'min_vv_by_vh',  'max_vv_by_vh', 'range_vv_by_vh', 'mean_vv_by_vh', 'correlation_vv_by_vh', 'permutation_entropy_vv_by_vh',
                                                 ##'min_rvi','max_rvi','range_rvi','mean_rvi'
                                                 ] )

## Join Dataframes

In [78]:
def combine_two_datasets(dataset1,dataset2):
    '''
    Returns a  vertically concatenated dataset.
    Attributes:
    dataset1 - Dataset 1 to be combined 
    dataset2 - Dataset 2 to be combined
    '''
    data = pd.concat([dataset1,dataset2], axis=1)
    return data

In [88]:
crop_data = combine_two_datasets(crop_yield_data,features_data)
crop_data.to_csv("crop_data.csv",index = False)

In [89]:
crop_data = pd.read_csv("crop_data.csv")

In [90]:
# specify the columns to be dropped
columns_to_drop = ['District','Latitude', 'Longitude', 'Date of Harvest','Season(SA = Summer Autumn, WS = Winter Spring)', 'Rice Crop Intensity(D=Double, T=Triple)']
# drop the columns from the original dataframe
crop_data = crop_data.drop(columns=columns_to_drop)

In [91]:
def find_asset_by_band_common_name(item, common_name):
    for asset in item.assets.values():
        asset_bands = eo.ext(asset).bands
        if asset_bands and asset_bands[0].common_name == common_name:
            return asset
    raise KeyError(f"{common_name} band not found")

In [99]:
def read_band(href):
    with rasterio.open(href) as ds:
        aoi_bounds = ft.bounds(area_of_interest)
        warped_aoi_bounds = warp.transform_bounds("epsg:4326", ds.crs, *aoi_bounds)
        aoi_window = windows.from_bounds(transform=ds.transform, *warped_aoi_bounds)
        return ds.read(1, window=aoi_window)

In [106]:
def get_data_landsat8_sentinel1(dataframe_crop_data):
    
    catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
    
    box_size_deg = 0.001 # Surrounding box in degrees
        
    lat = dataframe_crop_data["Latitude"]
    lon = dataframe_crop_data["Longitude"]
    centroid = lat, lon
    
    # Find the min and max values for latitude and longitude
    min_lon = centroid[1]-box_size_deg/2
    min_lat = centroid[0]-box_size_deg/2
    max_lon = centroid[1]+box_size_deg/2
    max_lat = centroid[0]+box_size_deg/2
    bounds = (min_lon, min_lat, max_lon, max_lat)
    
    area_of_interest = {
        "type": "Polygon",
        "coordinates": [                [
                [bounds[0], bounds[1]],
                [bounds[2], bounds[1]],
                [bounds[2], bounds[3]],
                [bounds[0], bounds[3]],
                [bounds[0], bounds[1]],
            ]
        ],
    }
    
    start_date = datetime.datetime(2021, 11, 1)
    end_date = datetime.datetime(2022, 8, 31)

    # Define the date of interest
    date_of_interest = datetime.datetime(2021, 5, 15)

    # Initialize an empty list to store the items
    items = []

    # Loop through each date in the range
    for date in pd.date_range(start=start_date, end=end_date):
        # Convert the date to the format required by the API
        date_str = date.strftime('%Y-%m-%d')

        # Search for items for the current date
        search = catalog.search(
            collections=["landsat-8-c2-l2"],
            intersects=area_of_interest,
            datetime=date_str,
            query={"eo:cloud_cover": {"lt": 10}},
        )

        # Add the items to the list
        items.extend(search.get_items())

    # Calculate the difference between the acquisition dates and the date of interest
    differences = [abs(datetime.datetime.strptime(item.properties['datetime'], '%Y-%m-%dT%H:%M:%S.%fZ') - date_of_interest) for item in items]

    # Sort the items by the differences
    sorted_items = [item for _, item in sorted(zip(differences, items))]

    # Select the item with the smallest difference
    selected_item = sorted_items[0]

    # Sort the items by cloud cover
    items_sorted_by_cloud_cover = sorted(items, key=lambda item: eo.ext(item).cloud_cover)

    # Select the item with the smallest cloud cover
    selected_item = items_sorted_by_cloud_cover[0]

    r = read_band(
        pc.sign(find_asset_by_band_common_name(selected_item, "red").href)
    ).astype(float)

    nir = read_band(
        pc.sign(find_asset_by_band_common_name(selected_item, "nir08").href)
    ).astype(float)
    
        
    # calculate NDVI
    ndvi = (nir - r) / (nir + r)
    
    # calculate SATVI
    L = 0.5
    satvi = (nir - r)
    
    return ndvi, satvi

In [107]:
# Define the function to retrieve the satellite data
def get_landsat8_satellite_data(row):
    ndvi, satvi = get_data_landsat8_sentinel1(row)
    ndvi_series = pd.Series(np.ravel(ndvi), index=['NDVI_{}'.format(i) for i in range(len(ndvi.ravel()))])
    satvi_series = pd.Series(np.ravel(satvi), index=['SATVI_{}'.format(i) for i in range(len(satvi.ravel()))])
    return pd.concat([ndvi_series, satvi_series])


In [108]:
# Apply the function to each row of crop_data
ndvi_satvi_df = crop_data.apply(get_landsat8_satellite_data, axis=1)

# Add the new columns to the crop_data DataFrame
crop_data = pd.concat([crop_data, ndvi_satvi_df], axis=1)

APIError: <html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>nginx</center>
</body>
</html>


In [36]:
from sklearn.decomposition import PCA

# Select only the Landsat 8 data columns
landsat8_data = crop_data.filter(regex='^(NDVI|SATVI)_')

# Instantiate a PCA object with the desired number of components
n_components = 10
pca = PCA(n_components=n_components)

# Fit the PCA model to the data
pca.fit(landsat8_data)

# Transform the data into the principal component space
landsat8_pca = pca.transform(landsat8_data)

# Create a new DataFrame with the principal components
pc_df = pd.DataFrame(data=landsat8_pca, columns=[f'PC{i+1}' for i in range(n_components)])

# Merge the original DataFrame with the principal component DataFrame
crop_data_pca = pd.concat([crop_data.drop(landsat8_data.columns, axis=1), pc_df], axis=1)


In [61]:
# specify the columns to be dropped
columns_to_drop = ['District','Latitude', 'Longitude', 'Date of Harvest','Season(SA = Summer Autumn, WS = Winter Spring)','Rice Crop Intensity(D=Double, T=Triple)','Date of Harvest']

# drop the columns from the original dataframe
crop_data_pca = crop_data_pca.drop(columns=columns_to_drop)

## Model Building

In [92]:
X = crop_data.drop(columns=['Rice Yield (kg/ha)']).values
y = crop_data['Rice Yield (kg/ha)'].values
# Choose any random state

## GradientBoostingRegressor 

In [93]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler


def objective(trial):
    # Load your data here
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


    # Define the hyperparameters to optimize
    n_estimators = trial.suggest_int("n_estimators", 100, 1000, step=100)
    max_depth = trial.suggest_int("max_depth", 1, 10)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)
    max_features = trial.suggest_categorical("max_features", ["auto", "sqrt", "log2", None])

    # Create the GradientBoostingRegressor
    gbr = GradientBoostingRegressor(n_estimators=n_estimators,
                                     max_depth=max_depth,
                                     learning_rate=learning_rate,
                                     min_samples_split=min_samples_split,
                                     min_samples_leaf=min_samples_leaf,
                                     max_features=max_features,
                                     random_state=42)

    # Fit the model and make predictions
    gbr.fit(X_train, y_train)
    y_pred = gbr.predict(X_test)

    # Calculate the mean
    score = r2_score(y_test,y_pred)
    return score


In [94]:
# Create the study object and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=500)

[I 2023-03-30 23:53:49,792] A new study created in memory with name: no-name-8d61cea8-024b-4842-ae00-2282b78467c2
[I 2023-03-30 23:53:49,879] Trial 0 finished with value: 0.549946430660157 and parameters: {'n_estimators': 100, 'max_depth': 4, 'learning_rate': 0.08861274783719836, 'min_samples_split': 3, 'min_samples_leaf': 2, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.549946430660157.
[I 2023-03-30 23:53:53,574] Trial 1 finished with value: 0.30590609793296397 and parameters: {'n_estimators': 900, 'max_depth': 10, 'learning_rate': 0.00036192299611185883, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': None}. Best is trial 0 with value: 0.549946430660157.
[I 2023-03-30 23:53:53,893] Trial 2 finished with value: 0.5873958868499105 and parameters: {'n_estimators': 400, 'max_depth': 1, 'learning_rate': 0.052353675703863733, 'min_samples_split': 6, 'min_samples_leaf': 5, 'max_features': 'auto'}. Best is trial 2 with value: 0.5873958868499105.
[I 2023-03-30 23:53:

In [95]:
# Print the best hyperparameters and objective value found
print('Best trial: score {},\nparams {}'.format(study.best_trial.value, study.best_trial.params))

Best trial: score 0.6222127231339081,
params {'n_estimators': 600, 'max_depth': 1, 'learning_rate': 0.005101798595564616, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'auto'}


In [97]:
 gbr = GradientBoostingRegressor(    n_estimators=600,
                                     max_depth=1,
                                     learning_rate=0.005101798595564616,
                                     min_samples_split=5,
                                     min_samples_leaf=1,
                                     max_features = 'auto',
                                     random_state=42)

# Fit the model and make predictions
gbr.fit(X, y)

GradientBoostingRegressor(learning_rate=0.005101798595564616, max_depth=1,
                          max_features='auto', min_samples_split=5,
                          n_estimators=600, random_state=42)

## Test File

In [82]:
test_file = pd.read_csv('Challenge_2_submission_template.csv')

In [46]:
## Get Sentinel-1-RTC Data

num_parts = 10
train_band_values_parts = np.array_split(test_file, num_parts)


assets = ['vh','vv']
results = []

for part in train_band_values_parts:
    part_results = part.apply(lambda x: get_sentinel_data(x['Longitude'], x['Latitude'],x['Season(SA = Summer Autumn, WS = Winter Spring)'],x['Rice Crop Intensity(D=Double, T=Triple)'],x['Date of Harvest'],assets), axis=1)
    results.append(part_results)

# Concatenate the results
submission_vh = []
submission_vv = []
submission_vv_by_vh = []

for r in results:
    submission_vh += [x[0] for x in r]
    submission_vv += [x[1] for x in r]
    submission_vv_by_vh += [x[2] for x in r]

submission_vh_vv_data = pd.DataFrame(list(zip(submission_vh,submission_vv,submission_vv_by_vh)),columns = ["vv_list","vh_list","vv/vh_list"])


In [99]:
# Generating Statistical Features for VV,VH and VV/VH and creating a dataframe
features_s = generate_stastical_features(submission_vh_vv_data)

In [111]:
submission_features = combine_two_datasets(test_file,features_s)

In [112]:
# specify the columns to be dropped
columns_to_drop = ['ID No','Predicted Rice Yield (kg/ha)','District','Latitude', 'Longitude', 'Date of Harvest','Season(SA = Summer Autumn, WS = Winter Spring)', 'Rice Crop Intensity(D=Double, T=Triple)']
# drop the columns from the original dataframe
submission_features = submission_features.drop(columns=columns_to_drop)

In [113]:
#Making predictions
final_predictions = gbr.predict(submission_features.values)
final_prediction_series = pd.Series(final_predictions)
#Combining the results into dataframe
test_file['Predicted Rice Yield (kg/ha)']=list(final_prediction_series)
#Dumping the predictions into a csv file.
test_file.to_csv("challenge_2_submission_rice_crop_yield_prediction.csv",index = False)